pmgus_dev_stuff.ipynb

In [1]:
import pandas as pd
import numpy as np
import requests
import os

In [2]:
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

get pre-market price data for SOFI
get pre-market clone data of TradingView
add features new and other ideas that we find